In [1]:
import numpy as np
import sympy as sp
from sympy.abc import x, y
import matplotlib.pyplot as plt
import pickle
from datetime import datetime
from functools import reduce
from scipy.optimize import fsolve
from IPython.display import clear_output

In [3]:
with open("data/NE_3-14.pk", 'rb') as file:
    data = pickle.load(file)

In [4]:
full_soln = lambda x: list(x) + [1 - sum(x[1:])]

N_min = 3
for N in range(15, 31):
    start_time = datetime.now()
    print("N = ", N)
    print("Start Time: ", start_time)
    
    p = sp.IndexedBase('p')
    i = sp.symbols('i', cls=sp.Idx, range=(1,N))
    Z0, c0 = sp.symbols('Z0, c0')
    Z0 = sp.summation(p[i], (i, 1, N))**(N-1)
    L = lambda i: lambda Q: p[i]*sp.diff(Q, p[i]).subs(p[i], 0)
    OneMinusL = lambda i: lambda Q: Q - L(i)(Q)
    
    Z_memo = dict()
    def Z(i):
        if i == 0:
            return Z0
        if i in Z_memo.keys():
            return Z_memo[i]
        Z_memo[i] = OneMinusL(i)(Z(i-1))
        return Z_memo[i]

    c_memo = dict()
    def c(i):
        if i in c_memo.keys():
            return c_memo[i]
        c_memo[i] = Z(i-1).subs(p[i], 0)
        return c_memo[i]
    
    func = lambda x: [sp.lambdify([c0]+[p[i] for i in range(1, N)], c(j).subs(p[N], 1-sp.summation(p[i], (i, 1, N-1)))-c0)(*x) for j in range(1, N+1)]
    soln = full_soln(fsolve(func, [1/N for _ in range(N)] if N == N_min else data[N-1]))
    data[N] = soln
    print(soln)
    
    with open("data/test/NE{}.pk".format(N), 'wb') as file:
        pickle.dump(data, file)

    end_time = datetime.now()
    print("End Time: ", end_time)
    diff = end_time - start_time
    print("Elapsed Time: {} days {} hours {} minutes {} seconds {} milliseconds {} microseconds".format(diff.days, diff.seconds // 3600, (diff.seconds // 60) % 60, diff.seconds % 60, diff.microseconds // 1000, diff.microseconds % 1000))
    print()

N =  15
Start Time:  2024-04-30 10:11:33.160137
[0.0619134691684164, 0.18021694243107564, 0.17234691049741163, 0.16223592309729187, 0.14871839429724848, 0.12990629877789203, 0.10328212646844286, 0.06786880379737693, 0.029774834890986486, 0.005476181597898569, 0.00017341336686419527, 1.707896832623507e-07, -1.222646639286061e-12, -2.0758583734207493e-12, 1.9591014831574416e-13, -9.069189843557979e-12]
End Time:  2024-04-30 22:52:43.985383
Elapsed Time: 0 days 12 hours 41 minutes 10 seconds 825 milliseconds 246 microseconds

N =  16
Start Time:  2024-04-30 22:52:43.985383


KeyboardInterrupt: 